In [ ]:
include("./fermionise.jl")
include("./models.jl")
include("./quantuminfo.jl")

In [ ]:
function main(numBath)
    basisStates = BasisStates(2 * (1 + numBath), 1 + numBath, 0.)
    hamiltonian = KondoHamiltonian(basisStates, numBath, [[1], 10])
    E, X = diagonalise(basisStates, hamiltonian)
    EE_imp = EntanglementEntropy(X[1], [1, 2])
end

main(1)